## [Draft notebook for April 2021 Earthdata Webinar]

#### Learning Objectives (tbd)
* (goal from discussion) Introduce audience to the “Ice cubed” (ICESat/OIB/ICESat-2) missions and products
* (goal from discussion Promote icepyx and IceFlow as a way to address the need to access/harmonize multiple data products/resolutions/formats/structure


#### Webinar Outline
* Intro of harmonization challenge
    - Moving one level deeper into the need for time series analysis across disparate platforms
* Intro to the altimetry missions
    - Use the IceFlow intro notebook to guide this
    - Highlight the impressive time series
* Overview of IceFlow and icepyx
    - Audience and use cases of these tools
    - Icepyx genesis, open science and community development principles
        - working on that balance between community development and end use
    - IceFlow overview
    - How do these tools address challenges
        - IceFlow aims to make it easier to harmonize OIB/ICESat data with ICESat-2 using backend IceFlow API
* Use Case / Application
    - Leverage ICESat-2 hackweek topics
    - https://github.com/ICESAT-2HackWeek/geospatial-analysis/blob/master/shean_ICESat-2_hackweek_tutorial_GeospatialAnalysis.ipynb
* Short navigation of nsidc site, how to get to tools, notebook
    - Have Jennifer post links while we talk.
* Demos
    - **Use this notebook to pulls pieces of iceflow / icepyx using a specific science application**

# MAKING CHANGES

# Changes